In [ ]:
from pyspark.sql.types import StringType
import json 
import pandas as pd
from sklearn.linear_model import LogisticRegression

# build a logistic regression model 
gamesDF = pd.read_csv("https://github.com/bgweber/Twitch/raw/master/Recommendations/games-expand.csv")
model = LogisticRegression() 
model.fit(gamesDF.iloc[:,0:10], gamesDF['label'])

# define the UDF for scoring users 
def score(row):
    d = json.loads(row)
    p = pd.DataFrame.from_dict(d, orient = "index").transpose()        
    pred = model.predict_proba(p.iloc[:,0:10])[0][0]
    result = {'User_ID': d['User_ID'], 'pred': pred }
    return str(json.dumps(result))

# read from Kafka 
df = spark.readStream.format("kafka") 
  .option("kafka.bootstrap.servers", "{aws_external_ip}:9092") 
  .option("subscribe", "dsp").
  .option("startingOffsets", "earliest").load()

# select the value field and apply the UDF     
df = df.selectExpr("CAST(value AS STRING)")
score_udf = udf(score, StringType())    
df = df.select( score_udf("value").alias("value"))

# Write results to Kafka 
query = df.writeStream.format("kafka") 
  .option("kafka.bootstrap.servers", "{aws_external_ip}:9092") 
  .option("topic", "predictions") 
  .option("checkpointLocation", "/temp").start()

In [ ]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer('predictions',
     bootstrap_servers=['{aws_external_ip}:9092'],
     value_deserializer=lambda x: loads(x))

for x in consumer:
    print(x.value)